In [2]:
import warnings 
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import datetime as dt
import copy
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Background/Ideas

- Features will be based off seasonal,weekly,career, and last 5 based values
- Idea is that certain players which can be differentiated by career based values paired with weekly performance and or seasonal (team strength proxy) can be paired to build something relatively predictive.
- Interactivity can be dependent on clicking and choosing assortment of players and identifying/projecting current projections.
- Data seems to get updated weekly so these predictions would change over time as well.

# Data

This section focuses on pulling the data and prepping/aggregating the dependent variable. (Fantasy Points)

In [ ]:
roster_data = nfl.import_seasonal_rosters([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999])
pbp_df = pd.DataFrame(nfl.import_pbp_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
# injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000]))
schedules_df = pd.DataFrame(nfl.import_schedules([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

# Base Data Transformations

- Getting General RB and WR stats and creating complete dataframe.

In [ ]:
## Basic PBP Passing Stats

def get_opposing_team(df):
    if df['home_team'] == df['team']:
        val = df['away_team']
    elif df['away_team'] == df['team']:
        val = df['home_team']
    else:
        val = None

    return val




# team = roster_data[roster_data['depth_chart_position'].isin(['RB','WR'])][['season','player_id','team','depth_chart_position']]

team = roster_data[['season','player_id','team','depth_chart_position']]

receiver_rusher_stats =  pbp_df[(pbp_df['receiver_player_id'].notnull()) | (pbp_df['rusher_player_id'].notnull())]
                         

receiver_rusher_stats['two_points'] = np.where(receiver_rusher_stats['two_point_conv_result'] == 'success',1,0)
                         
receiver_rusher_stats.rename(columns = {'complete_pass':'reception'},inplace = True)



receiver_stats= receiver_rusher_stats.groupby(['game_id', 'game_date', 'week','div_game','posteam','defteam', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'receiver_player_id', 'receiver_player_name','season']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'reception': 'sum',
    'interception': 'sum', #the passing stats are duplicated for receivers
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch':'sum',
    'touchdown':'sum',
    'fumble': 'sum',
    'two_points': 'sum'
}).reset_index()


rushing_stats = receiver_rusher_stats.groupby(['game_id', 'game_date', 'week', 'div_game','posteam','defteam', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'rusher_player_id', 'rusher_player_name','season']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'reception': 'sum',
    'interception': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch': 'sum',
    'touchdown':'sum',
    'fumble': 'sum',
    'two_points': 'sum'
}).reset_index()


## Grabbing seasonal info


team['team'] = team['team'].replace({'OAK':'LV', 'STL':'LA', 'SD':'LAC','HST':'HOU', 'BLT':'BAL', 'CLV':'CLE','SL':'LA','ARZ':'ARI'})


# team.rename(columns = {'player_id':'passer_player_id'},inplace = True)

## Standardizing Columns
rushing_stats.rename(columns = {'rusher_player_id':'player_id'}, inplace = True)

receiver_stats.rename(columns = {'receiver_player_id':'player_id'}, inplace = True)

rushing_stats.rename(columns = {'rusher_player_name':'player_name'}, inplace = True)

receiver_stats.rename(columns = {'receiver_player_name':'player_name'}, inplace = True)


rusher_receiver_df = pd.concat([receiver_stats,rushing_stats])




# rusher_receiver_df = rusher_receiver_df.merge(team, on = ['player_id','season'], how = 'inner')


## Aggregate average score to opposition 

# rusher_receiver_df['opponent_team'] = rusher_receiver_df.apply(get_opposing_team,axis = 1)

# print('Number Missing Opponent:' + str(rusher_receiver_df[rusher_receiver_df['opponent_team'].isna()].shape[0]))
# rusher_receiver_df = rusher_receiver_df[~rusher_receiver_df['opponent_team'].isna()]





game_score_info = schedules_df[['season','home_score','away_score','game_id']].copy()




rusher_receiver_df = rusher_receiver_df.merge(game_score_info, on = ['game_id','season'], how = 'inner')

# rusher_receiver_df.rename(columns = {'defteam':'opponent_team'}, inplace = True)


# rusher_receiver_df = rusher_receiver_df.groupby(['game_id', 'game_date', 'week', 'div_game', 'posteam','defteam','home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'player_id', 'player_name','season','home_score','away_score','team','depth_chart_position','opponent_team']).agg({
#     'passing_yards': 'sum',
#     'air_yards': 'sum',
#     'pass_touchdown': 'sum', 
#     'pass_attempt': 'sum',
#     'reception': 'sum',
#     'interception': 'sum',
#     'rush_attempt': 'sum',
#     'rushing_yards': 'sum',# Sum passing yards
#     'rush_touchdown': 'sum',
#     'lateral_rush': 'sum',
#     'receiving_yards': 'sum',
#     'yards_after_catch': 'sum',
#     'touchdown':'sum',
#     'fumble': 'sum',
#     'two_points': 'sum'
# }).reset_index()



rusher_receiver_df = rusher_receiver_df.groupby(['game_id', 'game_date', 'week', 'div_game', 'posteam','defteam','home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'player_id', 'player_name','season']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'reception': 'sum',
    'interception': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch': 'sum',
    'touchdown':'sum',
    'fumble': 'sum',
    'two_points': 'sum'
}).reset_index()



rusher_receiver_df.rename(columns = {'defteam':'opponent_team'} , inplace = True )


# #Checking the passing stats dataframe
# rusher_receiver_df.head(2)

In [ ]:

rusher_receiver_df.shape

# Creating Fantasy Points Column


Offensive Players:

- Passing Yards: 1 point per 25 yards
- Passing Touchdowns: 4 points
- Passing Interceptions: -2 points
- Rushing Yards: 1 point per 10 yards
- Rushing Touchdowns: 6 points
- Receptions: 1 points (only if using PPR scoring)
- Receiving Yards: 1 point per 10 yards
- Receiving Touchdowns: 6 points
- 2-Point Conversions: 2 points
- Fumbles Lost: -2 points
- Fumble Recovered for a Touchdown: 6 points

In [ ]:
rusher_receiver_df['fantasy_points'] = ((rusher_receiver_df['passing_yards']/25 )
                                         + (rusher_receiver_df['pass_touchdown'] * 4) + 
                                         (rusher_receiver_df['interception'] * -2) +
                                         (rusher_receiver_df['reception'] * 1) +
                                         (rusher_receiver_df['touchdown'] * 6) +
                                         (rusher_receiver_df['receiving_yards'] * .1) +
                                         (rusher_receiver_df['fumble'] * -2) +
                                         (rusher_receiver_df['two_points'] * 2))

In [ ]:
rusher_receiver_df.head(5)['fantasy_points']

# Last 5

In [ ]:

df_rusher_receiver_game_level = rusher_receiver_df.groupby(['game_id', 'game_date', 'week', 'season', 'posteam', 'opponent_team', 'player_name', 'player_id']).agg({
    # Game level
    'home_team': 'first',
    'away_team': 'first',

    # Play level
    'fantasy_points': 'sum',
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'reception': 'sum',
    'interception': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch': 'sum',
    'touchdown':'sum',
    'fumble': 'sum',
    'two_points': 'sum'

})

df_rusher_receiver_game_level["home"] = df_rusher_receiver_game_level["home_team"] == df_rusher_receiver_game_level.index.get_level_values("posteam")
df_rusher_receiver_game_level.drop(columns=['home_team', 'away_team'], inplace=True)

In [ ]:
def calc_agg_stats(group, fields, career=True):
    # Create a copy to avoid modifying the original
    # df = pd.DataFrame({'game_date': group['game_date']}, index=group.index)
    df = pd.DataFrame(index=group.index)
    
    # Sort chronologically
    group_sorted = group.sort_values('game_date')

    # Calculate the number of unique games for career, season, and prior season
    if career:
        df['n_games_career'] = range(len(group_sorted))

    df['n_games_season'] = group_sorted.groupby(
        group_sorted.index.get_level_values('season')
    ).cumcount()

    # df['n_games_prior_season'] = group_sorted.groupby(
    #     group_sorted.index.get_level_values('season')
    # ).transform('size').shift()



    # Calculate aggregate stats
    for field in fields:
        if career:
            # Career stats
            df[f'{field}_mean_career'] = group_sorted[field].transform(lambda x: x.expanding().mean().shift())
            df[f'{field}_total_career'] = group_sorted[field].transform(lambda x: x.expanding().sum().shift())
        
        # Season stats
        df[f'{field}_mean_season'] = group_sorted.groupby([group_sorted.index.get_level_values('season')])[field].transform(lambda x: x.expanding().mean().shift())
        df[f'{field}_total_season'] = group_sorted.groupby([group_sorted.index.get_level_values('season')])[field].transform(lambda x: x.expanding().sum().shift())

        # # Prior season stats
        # df[f'{field}_mean_prior_season'] = group_sorted.groupby([group_sorted.index.get_level_values('season') - 1])[field].transform('mean')
        
        # Last 5 games
        df[f'{field}_mean_last5'] = group_sorted[field].transform(lambda x: x.rolling(window=5, min_periods=1).mean().shift())
        df[f'{field}_total_last5'] = group_sorted[field].transform(lambda x: x.rolling(window=5, min_periods=1).sum().shift())
        # Last Game
        df[f'{field}_last'] = group_sorted[field].shift()
    return df

In [ ]:
fields = ['fantasy_points','reception','rushing_yards','touchdown','receiving_yards','fumble','passing_yards','pass_touchdown','two_points']


# Apply the function
df_rusher_receiver_game_level = df_rusher_receiver_game_level.groupby(['player_name', 'player_id']).apply(calc_agg_stats, fields=fields)



In [ ]:
df_rusher_receiver_game_level = df_rusher_receiver_game_level.reset_index(0).reset_index(0).drop(columns = ['player_name','player_id']).reset_index()

# Opponent Last Scored

In [ ]:
schedules_df_copy = schedules_df[schedules_df['game_id'].isin(schedules_df['game_id'].unique()) & (schedules_df['gameday'] >= '2001-09-09')]
schedules_df_copy.rename(columns = {'gameday':'game_date'}, inplace = True)

home_teams = schedules_df_copy[['game_id', 'game_date','season','home_team','away_score','week']].copy()

away_teams = schedules_df_copy[['game_id', 'game_date','season','away_team','home_score','week']].copy()

home_teams.rename(columns = {'home_team':'team','away_score':'points_allowed'}, inplace = True)
away_teams.rename(columns = {'away_team':'team','home_score':'points_allowed'}, inplace = True)

points_allowed_df = pd.concat([home_teams,away_teams])

points_allowed_df = points_allowed_df.groupby(['game_id', 'game_date','season','week','team']).agg({'points_allowed':'sum'})

group_sorted = points_allowed_df.sort_values('week')

pa_df = group_sorted.groupby(['team']).apply(calc_agg_stats, fields=['points_allowed']).reset_index(0).drop(columns = 'team').reset_index()[['game_id','game_date','season','week','team','points_allowed_mean_season','points_allowed_mean_last5']]


pa_df.rename(columns = {'team':'opponent_team'}, inplace = True)

### Features from pa_df are the oppositions points allowed until a certain measure of time

In [ ]:

rusher_receiver_features = rusher_receiver_df.merge(df_rusher_receiver_game_level, how = 'inner' ,on = ['game_id','game_date','week','season','posteam','opponent_team','player_name','player_id'])
# rusher_receiver_features['opponent_team'] = np.where(rusher_receiver_features['team'] == rusher_receiver_features['home_team'],rusher_receiver_features['away_team'],rusher_receiver_features['home_team'])
rusher_receiver_features = rusher_receiver_features.merge(pa_df , how = 'inner',on = ['game_date','season','week','opponent_team','game_id'])


rusher_receiver_features = rusher_receiver_features.fillna(0)

# Last Game Stats

In [ ]:
rusher_receiver_features.shape

In [ ]:
df_combined = rusher_receiver_features.copy()

In [ ]:
# Calculate the percentage of null values in each column
null_percentages = rusher_receiver_features.isnull().mean() * 100

# Sort the percentages in descending order for better readability
null_percentages = null_percentages.sort_values(ascending=False)

# Format the output to display percentages with two decimal places
null_percentages_formatted = null_percentages.apply(lambda x: f"{x:.2f}%")

# Print the results
print("Percentage of Null Values in Each Column:")
print(null_percentages_formatted)





In [ ]:
# Ensure 'temp' and 'wind' are numeric
df_combined['temp'] = pd.to_numeric(df_combined['temp'], errors='coerce')
df_combined['wind'] = pd.to_numeric(df_combined['wind'], errors='coerce')

# Calculate mean 'temp' and 'wind' by stadium
temp_wind_means = (
    df_combined.groupby('stadium')[['temp', 'wind']]
    .mean()
    .reset_index()
)

# Merge the mean values back to the original DataFrame
df_combined = pd.merge(
    df_combined,
    temp_wind_means,
    on='stadium',
    how='left',
    suffixes=('', '_mean')
)

# Impute missing 'temp' and 'wind' with the group mean values
df_combined['temp'].fillna(df_combined['temp_mean'], inplace=True)
df_combined['wind'].fillna(df_combined['wind_mean'], inplace=True)

# If any missing 'temp' or 'wind' values remain, fill them with the overall mean
df_combined['temp'].fillna(df_combined['temp'].mean(), inplace=True)
df_combined['wind'].fillna(df_combined['wind'].mean(), inplace=True)

# Drop the temporary mean columns
df_combined.drop(columns=['temp_mean', 'wind_mean'], inplace=True)

# For the rest of the columns, fill missing values with 0
# Exclude 'temp' and 'wind' as they've already been imputed
columns_to_fill = df_combined.columns.difference(['temp', 'wind'])
df_combined[columns_to_fill] = df_combined[columns_to_fill].fillna(0)

# Check if any missing values remain
remaining_nulls = df_combined.isnull().sum()
if remaining_nulls.sum() > 0:
    print("Remaining null values after imputation:")
    print(remaining_nulls[remaining_nulls > 0])
else:
    print("All missing values have been imputed.")

In [ ]:
x_vars = ['home_team','away_team','spread_line',
 'n_games_career',
 'n_games_season',
 'fantasy_points_mean_career',
 'fantasy_points_total_career',
 'fantasy_points_mean_season',
 'fantasy_points_total_season',
 'fantasy_points_mean_last5',
 'fantasy_points_total_last5',
 'fantasy_points_last',
 'reception_mean_career',
 'reception_total_career',
 'reception_mean_season',
 'reception_total_season',
 'reception_mean_last5',
 'reception_total_last5',
 'reception_last',
 'rushing_yards_mean_career',
 'rushing_yards_total_career',
 'rushing_yards_mean_season',
 'rushing_yards_total_season',
 'rushing_yards_mean_last5',
 'rushing_yards_total_last5',
 'rushing_yards_last',
 'touchdown_mean_career',
 'touchdown_total_career',
 'touchdown_mean_season',
 'touchdown_total_season',
 'touchdown_mean_last5',
 'touchdown_total_last5',
 'touchdown_last',
 'receiving_yards_mean_career',
 'receiving_yards_total_career',
 'receiving_yards_mean_season',
 'receiving_yards_total_season',
 'receiving_yards_mean_last5',
 'receiving_yards_total_last5',
 'receiving_yards_last',
 'fumble_mean_career',
 'fumble_total_career',
 'fumble_mean_season',
 'fumble_total_season',
 'fumble_mean_last5',
 'fumble_total_last5',
 'fumble_last',
 'passing_yards_mean_career',
 'passing_yards_total_career',
 'passing_yards_mean_season',
 'passing_yards_total_season',
 'passing_yards_mean_last5',
 'passing_yards_total_last5',
 'passing_yards_last',
 'pass_touchdown_mean_career',
 'pass_touchdown_total_career',
 'pass_touchdown_mean_season',
 'pass_touchdown_total_season',
 'pass_touchdown_mean_last5',
 'pass_touchdown_total_last5',
 'pass_touchdown_last',
 'two_points_mean_career',
 'two_points_total_career',
 'two_points_mean_season',
 'two_points_total_season',
 'two_points_mean_last5',
 'two_points_total_last5',
 'two_points_last',
 'opponent_team',
 'points_allowed_mean_season',
 'points_allowed_mean_last5']

# Feature Selection

In [ ]:
from nfl_model import NFLModel


In [ ]:
def get_dummy_variables(df, drop_first=True, dummy_na=False):
    """
    Converts non-numerical columns in a DataFrame to dummy variables.

    Parameters:
    - df: pandas DataFrame
        The input DataFrame containing the data.
    - drop_first: bool, default=False
        Whether to drop the first level of categorical variables to avoid the dummy variable trap.
    - dummy_na: bool, default=False
        Add a column to indicate NaNs, if False NaNs are ignored.

    Returns:
    - df_dummies: pandas DataFrame
        The DataFrame with non-numeric columns converted to dummy variables.
    """
    # Identify non-numeric columns
    non_numeric_cols = df.select_dtypes(exclude=['number', 'bool']).columns.tolist()

    # If there are no non-numeric columns, return the original DataFrame
    if not non_numeric_cols:
        print("No non-numerical columns to convert.")
        return df.copy()

    # Convert categorical variables to dummy variables
    df_dummies = pd.get_dummies(df, columns=non_numeric_cols, drop_first=drop_first, dummy_na=dummy_na)

    return df_dummies

In [ ]:
columns_to_include = df_combined.columns.difference(['game_id', 'game_date', 'player_name'])
final_df = df_combined[x_vars + ['fantasy_points']].copy()
# final_df['player_id'] = final_df['player_id'].astype('category')
final_df = get_dummy_variables(final_df)

y_var = 'fantasy_points'

In [ ]:
# Initialize the model
model = NFLModel(data=final_df, target_variable=y_var)

# Preprocess data
model.preprocess_data()

# Perform feature selection
model.feature_selection()

# Evaluate models
model.evaluate_models()

# Get and print the results
results_df = model.get_results()
print(results_df)

# Scoreout Data

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

def preprocess_data(data,target_variable):
        """
        Preprocesses the data by splitting into training and testing sets,
        converting categorical variables to dummy variables, and scaling the features.
        """
        # Separate features and target
        X = data.drop(columns=[target_variable])
        y = data[target_variable]

        # Split data into training and testing sets
        X_train_raw, X_test_raw, y_train, y_test = train_test_split(
            X, y, test_size=.2, random_state= 42
        )

        # Align the training and testing data
        X_train, X_test = X_train_raw.align(X_test_raw, join='left', axis=1, fill_value=0)

        # Convert y_train and y_test to 1D arrays if necessary
        y_train = y_train.values.ravel()
        y_test = y_test.values.ravel()

        # Standardize data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)


        return X_train_scaled,X_test_scaled,y_train,y_test



In [ ]:
final_features = model.elastic_net_features

In [ ]:
x_train,x_test,y_train,y_test = preprocess_data(final_df, 'fantasy_points')


model = LinearRegression()

model.fit(x_train,y_train)


In [ ]:
# Predict train and test
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)


final_df['predictions'] = np.hstack([y_pred_train, y_pred_test])

In [ ]:
 
 final_df[['game_id',
 'game_date',
 'week',
 'div_game',
 'posteam',
 'opponent_team',
 'home_team',
 'away_team',
 'weather',
 'stadium',
 'spread_line',
 'total_line',
 'roof',
 'surface',
 'temp',
 'wind',
 'home_coach',
 'away_coach',
 'player_id',
 'player_name',
 'season']] = df_combined[['game_id',
 'game_date',
 'week',
 'div_game',
 'posteam',
 'opponent_team',
 'home_team',
 'away_team',
 'weather',
 'stadium',
 'spread_line',
 'total_line',
 'roof',
 'surface',
 'temp',
 'wind',
 'home_coach',
 'away_coach',
 'player_id',
 'player_name',
 'season']]

In [ ]:
final_df.to_csv('./rb_wr_predictions.csv')

## Performance

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error



In [ ]:
# preds = ebm.predict(X_test_raw)

# preds_df['actual'] = y_test['fantasy_points']



print("MAE: " + str(mean_absolute_error(y_pred_test, y_test)))

print("MSE: " + str(mean_squared_error(y_pred_test, y_test)))




## Interactive Viz ##

In [3]:
# !pip install bokeh_sampledata
import random
import math

In [61]:
from bokeh.models import Button,PreText,ColumnDataSource, MultiSelect, CustomJS, DataTable, TableColumn, Div, TabPanel, Tabs, Tooltip, FactorRange, Select, TextInput
from bokeh.plotting import figure, show, row, column
from bokeh.io import save, show, output_file
from bokeh.events import ButtonClick
from bokeh import palettes
src = pd.read_csv("rb_wr_predictions.csv").drop("Unnamed: 0", axis=1)
output_file("index.html", title = "Team 134: Fantasy Points Visualization")
src = src[src['season'] == 2024]
src['weeks'] = 'Week ' + src['week'].astype(str)
Viridis = palettes.Viridis256
colors1 = ['blue'] * len(src)
colors2 = ['red'] * len(src)
src['colors1'] = colors1
src['colors2'] = colors2
src['colors3'] = random.choices(Viridis, k=len(src))
src['predictions'] = src['predictions'].astype(float)
src['predictions'] = src['predictions'].round(1)
src['fantasy_points'] = src['fantasy_points'].round(1)
src = src.sort_values(by='predictions', ascending=False)
dest = src.copy()
dest.loc[:, ['predictions', 'fantasy_points','colors1', 'colors2']] = '' 
source = ColumnDataSource(src)
srctarget = dest
src2 = src.head(32)
dest2 = dest.head(32)
stateChange= ColumnDataSource(srctarget)
sc2 = ColumnDataSource(dest2)
source2 = ColumnDataSource(src2)
div2 = Div(text="""<h2 style="font-size:15px; color:black;">Data Table for Scatter Plots</h2>""",
                          sizing_mode="fixed", height=24, width=200)
div3 = Div(text="""<h2 style="font-size:15px; color:black;">Data Table for Bar Plots</h2>""",
                          sizing_mode="fixed", height=24, width=200)

TOOLS = "hover,pan,wheel_zoom,box_zoom,reset,save"

code_js = """ 
    function update() {
        var s1= sel1.value;
        var s2= sel2.options;
        for (const key in sc.data) {
            sc.data[key] = [];
        }
        for (const key in sc2.data) {
            sc2.data[key] = [];
        }
        if (s1 === undefined || s1.length == 0) {
            pr.text = "Please select a week";
            return;
        }
        pr.text = "";
        var lo = 0.0;
        var lo2 = 0.0;
        const pl = [];
        const pl2 = [];
        for (var i = 0; i < source.get_length(); i++) {
            if (s1 === 'All') {
                if (!(s2 === undefined || s2.length == 0)) {
                    if(s2.indexOf(source.data['player_name'][i]) >= 0) { 
                        if(sc.data['index'].indexOf(source.data['index'][i]) < 0) {
                            for (const col in source.data) {
                                sc.data[col].push(source.data[col][i]);                
                            }
                        }
                        if(sc2.get_length() < 32 && sc2.data['index'].indexOf(source.data['index'][i]) < 0) {
                            for (const col in source.data) {
                                sc2.data[col].push(source.data[col][i]);                
                            }
                        }
                    }
                    else {
                        if(sc.data['index'].indexOf(source.data['index'][i]) >= 0) {
                            var q = sc.data['index'].indexOf(source.data['index'][i]);
                            for (const col in source.data) {
                                sc.data[col].splice(q, 1);                
                            }
                            
                        }
                        if(sc2.data['index'].indexOf(source.data['index'][i]) >= 0) {
                            var q = sc2.data['index'].indexOf(source.data['index'][i]);
                            for (const col in source.data) {
                                sc2.data[col].splice(q, 1);                
                            }
                            
                        }
                    }
                }
                else {
                    if(sc.data['index'].indexOf(source.data['index'][i]) < 0) {
                        pl.push(source.data['player_name'][i]);
                        for (const col in source.data) {
                            sc.data[col].push(source.data[col][i]);                
                        }
                    }
                    if(sc2.get_length() < 32 && sc2.data['index'].indexOf(source.data['index'][i]) < 0) {
                        pl2.push(source.data['player_name'][i]);
                        for (const col in source.data) {
                            sc2.data[col].push(source.data[col][i]);                
                        }
                    }
                }
            } 
            else {
                if (s1 === source.data['weeks'][i]) {
                    if (!(s2 === undefined || s2.length == 0)) {
                        if(s2.indexOf(source.data['player_name'][i]) >= 0) { 
                            if(sc.data['index'].indexOf(source.data['index'][i]) < 0) {
                                for (const col in source.data) {
                                    sc.data[col].push(source.data[col][i]);                
                                }
                            }
                            if(sc2.get_length() < 32 && sc2.data['index'].indexOf(source.data['index'][i]) < 0) {
                                for (const col in source.data) {
                                    sc2.data[col].push(source.data[col][i]);                
                                }
                            }
                        }
                        else {
                            if(sc.data['index'].indexOf(source.data['index'][i]) >= 0) {
                                var q = sc.data['index'].indexOf(source.data['index'][i]);
                                for (const col in source.data) {
                                    sc.data[col].splice(q, 1);                
                                }
                                
                            }
                            if(sc2.data['index'].indexOf(source.data['index'][i]) >= 0) {
                                var q = sc2.data['index'].indexOf(source.data['index'][i]);
                                for (const col in source.data) {
                                    sc2.data[col].splice(q, 1);                
                                }
                                
                            }
                        }
                    }
                    else {
                        if(sc.data['index'].indexOf(source.data['index'][i]) < 0) {
                            pl.push(source.data['player_name'][i]);
                            for (const col in source.data) {
                                sc.data[col].push(source.data[col][i]);                
                            }
                        }
                        if(sc2.get_length() < 32 && sc2.data['index'].indexOf(source.data['index'][i]) < 0) {
                            pl2.push(source.data['player_name'][i]);
                            for (const col in source.data) {
                                sc2.data[col].push(source.data[col][i]);                
                            }
                        }
                    }                                    
                }
            }
        }
        var pl1 = [...new Set(pl)]; 
        var pll2 = [...new Set(pl2)];
        if (!(s2 === undefined || s2.length == 0)) {
            p1.x_range.factors = s2;
            p2.x_range.factors = s2;
        }
        else {
            p1.x_range.factors = pl1;
            p2.x_range.factors = pll2;
        }
        const inlo1 = Math.min(...sc.data['predictions'])
        const inlo2 = Math.min(...sc.data['fantasy_points'])
        const inlo3 = Math.min(...sc2.data['predictions'])
        lo = (lo > inlo1 && inlo1 < 0) ? inlo1 : 0;
        lo = (lo > inlo2 && inlo2 < 0) ? inlo2 : 0;
        lo2 = (lo2 > inlo3 && inlo3 < 0) ? inlo3 : 0;
        p1.y_range.start = lo;
        p2.y_range.start = lo2;
        p3.y_range.start = lo;
    }
    update();
    sc.change.emit();
    sc2.change.emit();
    
"""
#Dropdowns lists
sort_df1 = sorted(src['week'].unique().tolist())
sort_df1 = ['Week ' + str(i) for i in sort_df1]
sort_df1 = ['All'] + sort_df1
sort_df2 = sorted(src['player_name'].astype(str).unique().tolist())
#Selects players
multi_select_player = MultiSelect(title="Players to plot:", height = 200, \
    value=[], \
    options=sort_df2)
#Buttons and selects
multi_select_player2 = MultiSelect(title="Selected Players:", height = 200, \
    value=[], \
    options=[])
pret = PreText(text="", \
        width=200, styles={'color': 'red', 'font-size': '9pt'})
but1 = Button(label="Add", button_type="success")
but2 = Button(label="Remove", button_type="danger")
but3 = Button(label="Visualize", button_type="primary")
but1.js_on_event(ButtonClick, CustomJS(args=dict(t1 = multi_select_player, ms=multi_select_player2,tex = pret), code = """
            var a1 = t1.value;
            var pret = tex;
            var temp = [];
            for (const a of ms.options) {
                temp.push(a);       
            }
            if (a1 == "undefined" || a1.length == 0) {
                pret.text = "Select player name to add";
            }
            else {
                pret.text = "";
                for(const a of a1) {
                    if (temp.indexOf(a) < 0) {
                        temp.push(a);
                    }
                }
            }
            ms.options = temp;
        """
    ))
but2.js_on_event(ButtonClick, CustomJS(args=dict(ms=multi_select_player2,tex = pret), code = """
            var b1 = ms.value;
            var pret = tex;
            var temp = [];
            for (const a of ms.options) {
                temp.push(a);       
            }
            if (b1 == "undefined" || b1.length == 0) {
                pret.text = "Select player name to remove";
            }
            else {
                pret.text = "";
                for(const a of b1) {
                    if (temp.indexOf(a) >= 0) {
                        temp.splice(temp.indexOf(a), 1);
                    }
                }
            }
            ms.options = temp;
            ms.value = [];
        """
    ))
ti = TextInput(placeholder='Enter player_name'
               )
ti.js_on_change('value', CustomJS(args=dict(ds=sort_df2, s=multi_select_player),
                                 code="s.options = ds.filter(i => i.toLowerCase().includes(cb_obj.value.toLowerCase()));"))
select1 = Select(options= sort_df1, value = [], title = 'Week')
p1 = figure(tools=TOOLS, toolbar_location="above", width=1200, height = 725,x_range = FactorRange(factors=sorted(list(set(stateChange.data['player_name'])))),title="NFL Fantasy Points Predictions")
p1.toolbar.logo = "grey"
p1.background_fill_color = "#d6cdcb"
p1.xaxis.axis_label = "Players"
p1.yaxis.axis_label = "Prediction points"
p1.grid.grid_line_color = "white"
p1.xgrid.grid_line_dash = 'dashed'
p1.xaxis.major_label_orientation = math.pi/4
p1.ygrid.grid_line_dash = 'dotted'
p1.hover.tooltips = [
    ("Player name", "@player_name"),
    ("Predicted Point", "@predictions{1.1}"),
    ("Fantasy Point:", "@fantasy_points{1.1}"),
    ("Season Avg:", "@fantasy_points_mean_season"),
    ("Def Allowed", "@points_allowed_mean_season"),
    ("Passing Yards", "@passing_yards_total_season"),
    ("Passing TD", "@pass_touchdown_total_season"),
    ("Rushing Yards", "@rushing_yards_total_season"),
    ("Touchdown Mean Season", "@touchdown_mean_season"),
]
scatterplot = p1.scatter("player_name", "predictions", size=10, source=stateChange, legend_label = 'Predictions',
          color='colors1', line_color="black", alpha=0.6)
scatterplot2 = p1.scatter("player_name", "fantasy_points", size=10, source=stateChange, legend_label = 'Fantasy Points',
          color='colors2', line_color="black", alpha=0.6)

p1.legend.location = "top_left"
p1.legend.click_policy="hide"
p1.legend.label_text_font = "times"
p1.legend.label_text_font_style = "italic"
p1.legend.label_text_color = "navy"


p1.legend.background_fill_color = "#d6cdcb"
p1.legend.background_fill_alpha = 0.2

week_l = sorted(src['week'].unique().tolist())
week_l = ['Week ' + str(i) for i in week_l]
p3 = figure(tools=TOOLS, toolbar_location="above", width=1200, height = 725, x_range = week_l, title="NFL Fantasy Points Predictions")
p3.toolbar.logo = "grey"
p3.background_fill_color = "#d6cdcb"
p3.xaxis.axis_label = "Weeks"
p3.yaxis.axis_label = "Prediction points"
p3.grid.grid_line_color = "white"
p3.xgrid.grid_line_dash = 'dashed'
p3.xaxis.major_label_orientation = math.pi/4
p3.ygrid.grid_line_dash = 'dotted'
p3.hover.tooltips = [
    ("Player name", "@player_name"),
    ("Predicted Point", "@predictions{1.1}"),
    ("Fantasy Point:", "@fantasy_points{1.1}"),
    ("Season Avg:", "@fantasy_points_mean_season"),
    ("Def Allowed", "@points_allowed_mean_season"),
    ("Passing Yards", "@passing_yards_total_season"),
    ("Passing TD", "@pass_touchdown_total_season"),
    ("Rushing Yards", "@rushing_yards_total_season"),
    ("Touchdown Mean Season", "@touchdown_mean_season"),
]
spl = p3.scatter("weeks", "predictions", size=10, source=stateChange,legend_label = 'Predictions',
          color='colors1', line_color="black", alpha=0.6)
spl2 = p3.scatter("weeks", "fantasy_points", size=10, source=stateChange,legend_label = 'Fantasy Points',
          color='colors2', line_color="black", alpha=0.6)

p3.legend.location = "top_left"
p3.legend.click_policy="hide"
p3.legend.label_text_font = "times"
p3.legend.label_text_font_style = "italic"
p3.legend.label_text_color = "navy"

p3.legend.background_fill_color = "#d6cdcb"
p3.legend.background_fill_alpha = 0.2
player_list = list(sc2.data['player_name'])
p2 = figure(x_range = FactorRange(factors=list(set(sorted(player_list, key=lambda x: sc2.data['predictions'][player_list.index(x)], reverse=True)))), tools=TOOLS, toolbar_location="above", x_axis_label = 'Player Name', y_axis_label = 'Fantasy Points',  width=1200, height = 725)
p2.grid.grid_line_color = "white"
p2.toolbar.logo = "grey"
p2.background_fill_color = "#d6cdcb"
p2.xgrid.grid_line_dash = 'dashed'
p2.ygrid.grid_line_dash = 'dotted'

p2.xaxis.major_label_orientation = math.pi/4

p2.hover.tooltips = [
    ("Player name", "@player_name"),
    ("Proj Point", "@predictions{1.1}"),
    ("Actual Point:", "@fantasy_points{1.1}"),
    ("Season Avg:", "@fantasy_points_mean_season"),
    ("Def Allowed", "@points_allowed_mean_season"),
    ("Passing Yards", "@passing_yards_total_season"),
    ("Passing TD", "@pass_touchdown_total_season"),
    ("Rushing Yards", "@rushing_yards_total_season"),
    ("Touchdown Mean Season", "@touchdown_mean_season"),
]

# plot bar chart glyph
p2.vbar(x = 'player_name', top = 'predictions', width=0.3, color='colors3', source=sc2, alpha = 1.2)
callback = CustomJS(args=dict(source=source, sc=stateChange, sel1 = select1, sel2 = multi_select_player2, p1 = p1, p2 = p2,p3=p3, sc2=sc2,pr=pret), code = code_js)
but3.js_on_event(ButtonClick, callback)
#Group in tab panel
tabs = Tabs(tabs=[
    TabPanel(child=p1, title="Scatter Player Name", tooltip=Tooltip(content="Scatter player name Plot Visualize fantasy predicted points", position="bottom_center")),
    TabPanel(child=p3, title="Scatter Week", tooltip=Tooltip(content="Scatter week Plot Visualize fantasy predicted points", position="bottom_center")),
    TabPanel(child=p2, title="Vertical Bar", tooltip=Tooltip(content="Vbar Plot Visualize fantasy predicted points", position="bottom_center")),
])
columns = [
            TableColumn(field="player_name", title="Players_name"),
            TableColumn(field="week", title="Week"),
            TableColumn(field="season", title="Seasons"),
            TableColumn(field="fantasy_points", title="Fantasy points"),
            TableColumn(field="predictions", title="Prediction Points"),
        ]

# data table for scatter
dtab = DataTable(source=stateChange, columns=columns, width=400, height=400)
# data table for bar
columns = [
            TableColumn(field="player_name", title="Players_name"),
            TableColumn(field="week", title="Week"),
            TableColumn(field="season", title="Seasons"),
            TableColumn(field="fantasy_points", title="Fantasy points"),
            TableColumn(field="predictions", title="Prediction Points"),
        ]

# data table
dtab2 = DataTable(source=sc2, columns=columns, width=400, height=400)
d1 = Button(label="Download Data for Scatter", button_type="primary")
d2 = Button(label="Download Data for Bar Plot", button_type="primary")

downloadCode = """
    function convertToCSV(objArray) {
        console.log(typeof objArray);
        var array = objArray;
        var str = '';
        console.log(array.get_length());
        for(const col in array.data) {
            if (col === 'colors1') break;
            str = (col === 'weeks') ? str + col : str + col + ',';
        }
        str += '\\r\\n';         
        for (var i = 0; i < array.get_length(); i++) {
            var line = '';
            for (var index in array.data) {
                if (index === 'colors1') break; 
                if (line != '') line += ','
                var d = array.data[index][i];
                d = String(d);
                d = d.split(",").join("-"); 
                line += d;
            }
            str += line + '\\r\\n';
        }
        return str;
    }
    var data = source;
    var csv = convertToCSV(data);
    var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
    var link = document.createElement('a');
    link.href = URL.createObjectURL(blob);
    link.download = (togg === 'scatter') ? 'ScatterData.csv' : 'BarData.csv';
    link.click();
"""
d1.js_on_click(CustomJS(args=dict(source=stateChange,togg='scatter'), code=downloadCode))
d2.js_on_click(CustomJS(args=dict(source=sc2,togg='vbar'), code=downloadCode))

control = column(tabs,row(column(div2,dtab, d1), column(div3,dtab2, d2)))
layout=row(column(select1,ti,multi_select_player, pret,row(but1,but2),multi_select_player2, but3), row(control))
save(layout)  # save the plot
show(layout)